In [4]:
import sys

In [5]:
sys.path.append('../')

In [6]:
from final_utils import *
from ResNet import *
from ResNet_FC import *
from tensorflow.keras.models import load_model
root_path = generate_root_path()

In [7]:
# No need to specify "with gpu 1" when using this block
## USE THIS BLOCK IF YOU HAVE MORE THAN 1 GPU 
gpus = tf.config.list_physical_devices('GPU')
index = 0
tf.config.experimental.set_memory_growth(gpus[index], True)
# select the gpu you want to use
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.set_visible_devices(gpus[index], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


In [8]:
weights_path = '../models/Softmax/ResNet_FC/'
model = load_model(weights_path)

In [9]:
good_imgs_path = glob(os.path.join(root_path, 'Data', 'AesthAI', 'benchmark', 'images', 'good', '*'))
bad_imgs_path = glob(os.path.join(root_path, 'Data', 'AesthAI', 'benchmark', 'images', 'bad', '*'))

good_imgs_path_2 = glob(os.path.join(root_path, 'Data', 'AesthAI', 'benchmark2', 'images', 'good', '*'))
bad_imgs_path_2 = glob(os.path.join(root_path, 'Data', 'AesthAI', 'benchmark2', 'images', 'bad', '*'))

good_imgs_path_joint = glob(os.path.join(root_path, 'Data', 'AesthAI', 'benchmark_connected', 'images', 'good', '*'))
bad_imgs_path_joint = glob(os.path.join(root_path, 'Data', 'AesthAI', 'benchmark_connected', 'images', 'bad', '*'))

good_alm_imgs_path = glob(os.path.join(root_path, 'Data', 'AesthAI', 'alm', 'splitted', 'alm_bench', 'images', 'good', '*'))
bad_alm_imgs_path = glob(os.path.join(root_path, 'Data', 'AesthAI', 'alm', 'splitted', 'alm_bench', 'images', 'bad', '*'))

good_rand_imgs_path = glob(os.path.join(root_path, 'Data', 'AesthAI', 'Sven_custom_data', 'images', 'good', '*'))
bad_rand_imgs_path = glob(os.path.join(root_path, 'Data', 'AesthAI', 'Sven_custom_data', 'images', 'bad', '*'))

bad_coco_imgs_path = glob(os.path.join(root_path, 'Data', 'AesthAI', 'Coco', 'images', 'bad', '*'))

paths_bench = good_imgs_path + bad_imgs_path
labels_bench = np.concatenate((np.ones(len(good_imgs_path)), np.zeros(len(bad_imgs_path))))

paths_bench_2 = good_imgs_path_2 + bad_imgs_path_2
labels_bench_2 = np.concatenate((np.ones(len(good_imgs_path_2)), np.zeros(len(bad_imgs_path_2))))

paths_bench_joint = good_imgs_path_joint + bad_imgs_path_joint
labels_bench_joint = np.concatenate((np.ones(len(good_imgs_path_joint)), np.zeros(len(bad_imgs_path_joint))))

paths_alm_bench = good_alm_imgs_path + bad_alm_imgs_path
labels_alm_bench = np.concatenate((np.ones(len(good_alm_imgs_path)), np.zeros(len(bad_alm_imgs_path))))

paths_rand_bench = good_rand_imgs_path + bad_rand_imgs_path
labels_rand_bench = np.concatenate((np.ones(len(good_rand_imgs_path)), np.zeros(len(bad_rand_imgs_path))))

paths_coco_bench = bad_coco_imgs_path
labels_coco_bench = np.zeros(len(bad_coco_imgs_path))

In [10]:
def calc_acc(labels, predicted):
    return np.sum(np.array(labels) == np.argmax(np.array(predicted), axis=1)) / len(labels)

def predict_from_path(model, paths, resize_func=None, size=None, for_all=False, model_CNN=None, is_norm=False):
    #always requires list of paths
    predicted = []
    
    for i, path in enumerate(paths):
        img_mg = read_img(path=path, resize_func=resize_func, size=size, for_all=for_all)
        pred_score = model.predict(img_mg, verbose=0)
        predicted.append(pred_score)
    
    predicted = np.array(predicted)
    predicted = np.squeeze(predicted)
        
    return predicted

### Prediction on benchmark

In [18]:
predicted = predict_from_path(model, paths_bench, resize_func=resize_add_border, size=(600, 600))

In [19]:
acc = calc_acc(labels_bench, predicted)
print(f'Accuracy: {acc}')

Accuracy: 0.7387387387387387


In [20]:
# Precision, Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_bench, pred)

Precision: 80.0 %
Recall: 51.06 %


### Prediction on benchmark 2

In [14]:
predicted = predict_from_path(model, paths_bench_2, resize_func=resize_add_border, size=(600, 600))

In [15]:
acc = calc_acc(labels_bench_2, predicted)
print(f'Accuracy: {acc}')

Accuracy: 0.7456479690522244


In [16]:
# Precision, Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_bench_2, pred)

Precision: 91.19 %
Recall: 56.18 %


### Prediction on joint benchmark 

In [ ]:
predicted = predict_from_path(model, paths_bench_joint, resize_func=resize_add_border, size=(600, 600))

In [ ]:
acc = calc_acc(labels_bench_joint, predicted)
print(f'Accuracy: {acc}')

In [ ]:
# Precision, Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_bench_joint, pred)

### Prediction on COCO

In [ ]:
predicted = predict_from_path(model, paths_coco_bench, resize_func=resize_add_border, size=(600, 600))

In [ ]:
acc = calc_acc(labels_coco_bench, predicted)
print(f'Accuracy: {acc}')

### Prediction on alm benchmark

In [ ]:
predicted = predict_from_path(model, paths_alm_bench, resize_func=resize_add_border, size=(600, 600))

In [ ]:
acc = calc_acc(labels_alm_bench, predicted)
print(f'Accuracy: {acc}')

In [ ]:
# Precision, Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_alm_bench, pred)

### Prediction on Sven custom data

In [ ]:
predicted = predict_from_path(model, paths_rand_bench, resize_func=resize_add_border, size=(600, 600))

In [ ]:
acc = calc_acc(labels_rand_bench, predicted)
print(f'Accuracy: {acc}')

In [ ]:
# Precision, Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_rand_bench, pred)